**Configuración**

Si está ejecutando este notebook en Google Colab, ejecute la siguiente celda para instalar las bibliotecas que necesitamos:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install transformers datasets
!pip install transformers==4.28.0 datasets

**Cargar dataset**

In [ ]:
import pandas as pd
data = pd.read_excel('/content/drive/Shareddrives/MeIA-Análisis de Sentimiento/training.xlsx', index_col=0)
#data.head()

Análizis de datos del conjunto de entrenamiento original.

In [ ]:
print("Número de instacias: ",len(data))
print("Distribución de instancias por clase:")
print(data["Class"].value_counts())

Número de instacias:  17500
Distribución de instancias por clase:
5    5250
4    4375
3    3500
2    2625
1    1750
Name: Class, dtype: int64


In [ ]:
data.head()

,Class
Review,
Dar mantenimiento adecuado y aclarar jurisdicción de quién es. De uno de los puentes que lleva a las cascadas cayó una turista 100 m hacia abajo en abril 2021 (obviamente falleció al caer sobre las rocas) y nunca se aclaró si la responsabilidad fue de CFE o del Municipio.,1
"yo estaba tan emocionada de este viaje, pero tras el 2-1 2 horas de autobús /viaje y 1 parada de un parque ecológico con un Cenote - me di cuenta este viaje era más de una trampa para turistas que un sitio histórico. Las ruinas en sí tienen más vendedores ladrando y haciendo ruido con sus mercancías que puede debilitar un palo. que siga usted y su guía y tratar de vender a usted mientras usted está tratando de escuchar al guía sobre lo que estás viendo. Hacía calor, y el poquito de sombra disponible fue tomado por los vendedores. La única cosa que he oído en ese viaje fue ""oferta! oferta! más barato que Wal-Mart! Casi gratis!"" que necesitan para mantener a los vendedores en una zona y para permitir a los visitantes que se acerquen a ellos, no sentimos acosados y seguido por ellos. Y por amor de Dios, acondicionar algunos lugares sombreados para sentarse allí!!! Había basura en las ruinas. . Nunca recomendaría este viaje a cualquiera. Fue un día entero y $ nunca volver. Una de las ""maravillas"" del mundo? Realmente? sólo me"" maravilla"" ¿por qué me dejaron lo que podría ser un sitio guay conseguir así crapped!.",1
"Este hotel es un engaño. De cuatro estrellas nada. Una estrella siendo generoso. Carece de higiene básica, el servicio pésimo, las habitaciones viejas, cama rota, polvo en todos lados, el desayuno impresentable, manteles sucios, sin servilletas, un desastre. Nunca más.",1
"La ducha se inundo completamente a los 5 min de usarla por una coladera sucia y el agua se transfirio al resto del baño. No habia toallas tampoco. Lo que habla de la poca higiene y poco cuidado a los detalles. El personal ni siquiera se disculpo por el incidente. Los ruidos en el pasillo se oyen muchísimo a la habitación. Mala experiencia y una noche por hab. Estandar costo casi 2000 pesos mexicanos, La decoracion se ve vieja y anticuada.",1
"Decidimos alquilar un taxi por un día para ir a Habana. El taxista recomendó y nos llevó a este restaurante para el almuerzo. Creo que él hizo esto como un montón de grupos de turistas ir allí. Para dos adultos y un niño de 7...y mi hijo de 9 años pagamos casi $150.00 canadiense para el almuerzo! No nos dimos cuenta los ""lados"" eran extra. La camarera Camino venta sugestiva ... sabiendo que los chicos es probable que se llena en el plato principal. Además, el 10% de propina era incluidos. La carne rallado que los niños lo estaba bien y abundante, pero no vale la pena los 18 dólares un plato. No podía comer la porción de langosta de mi comida como la langosta probado ... skunky ... creo que era un tipo diferente de langosta más barato que era asqueroso. Este fue un servicio de almuerzo, así que no había música en vivo ... quizás leer otras críticas, es donde este lugar es bueno para. La única cosa era la sabrosa bebida ... caña de azúcar y zumo de piña ... lo que estoy seguro de que podríamos conseguir por la mitad del precio en otro sitio. Fuera de todo nuestro viaje a Cuba, este restaurante fue la única decepción y empeoramiento del viaje! Casi pagado tanto para el almuerzo como nosotros pagamos el taxista que nos llevó desde Valedaro para todo el día!Más",1


**División estratificada**

Se divide estratificadamente el dataset original para formar un conjunto de entrenamiento y pruebas.

In [ ]:
from sklearn.model_selection import train_test_split
a,b = train_test_split(data, test_size=0.05, stratify = data['Class'])
print("Longitud del conjunto de entrenamiento (70%): ",len(a))
print("Longitud del conjunto de pruebas (30%): ",len(b))
print(len(a)+len(b))

Longitud del conjunto de entrenamiento (70%):  12250
Longitud del conjunto de pruebas (30%):  5250
17500


In [ ]:
print("Distribución de instancias del conjunto de entrenamiento por clase:")
print(a["Class"].value_counts())

Distribución de instancias del conjunto de entrenamiento por clase:
5    3675
4    3063
3    2450
2    1837
1    1225
Name: Class, dtype: int64


In [ ]:
from datasets import Dataset, concatenate_datasets, DatasetDict

ds_train = Dataset.from_pandas(a)
ds_test = Dataset.from_pandas(b)
ds_dict = {'train' : ds_train, 'test':ds_test}
dataset = DatasetDict(ds_dict)
dataset

DatasetDict({
    train: Dataset({
        features: ['Class', 'Review'],
        num_rows: 12250
    })
    test: Dataset({
        features: ['Class', 'Review'],
        num_rows: 5250
    })
})

**Pre-procesamiento del dataset**

Cambiar el nombre de la columana que contiene las clases a predecir por "labels"

In [ ]:
dataset = dataset.rename_column("Class", "labels")

In [ ]:
dataset

Las etiquetas (labels) en HF se inicializan a partir del numeró 0, por lo cuál, las etiquetas de polaridad se deben modifican de la siguiente manera: label 1 = 0, label 2 = 1, label 3 = 2, label 4 = 3 y label 5 = 4.

In [ ]:
def polarity_init(example):
  example["labels"] = example["labels"] - 1
  return example

dataset = dataset.map(polarity_init)

dataset.set_format("pandas")
df = dataset["train"][:]
print(df["labels"].value_counts())
dataset.reset_format()

Map:   0%|          | 0/12250 [00:00<?, ? examples/s]

Map:   0%|          | 0/5250 [00:00<?, ? examples/s]

4    3675
3    3063
2    2450
1    1837
0    1225
Name: labels, dtype: int64


**Login en Hugging Face para compartir el modelo**

In [ ]:
# Esto sólo funciona en Google Colab! Para los notebooks normales, es necesario ejecutar esto en el terminal
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 45, in main
    service.run()
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/c

Si no tienes instalado Git LFS, puedes hacerlo descomentando y ejecutando la celda de abajo:

In [ ]:
!apt install git-lfs
!git config --global user.email "vg055@hotmail.com"
!git config --global user.name "Victor055"

**Tokenizar las reseñas**

In [ ]:
from transformers import AutoTokenizer
model_checkpoint = "BSC-TeMU/roberta-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.vocab_size

50262

In [ ]:
def tokenize_reviews(examples):
    return tokenizer(examples["Review"], truncation=True)

columns = dataset["train"].column_names
columns.remove("labels")
encoded_dataset = dataset.map(tokenize_reviews, batched=True, remove_columns=columns)
encoded_dataset

Map:   0%|          | 0/12250 [00:00<?, ? examples/s]

Map:   0%|          | 0/5250 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 12250
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5250
    })
})

In [ ]:
encoded_dataset["train"][0]

**Cargar el modelo pre-entrenado**

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 5
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

**Definir metríca de evaluación**

In [ ]:
from datasets import load_metric

metric = load_metric("f1")
metric

<ipython-input-25-ee0dc6f5e028>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1")


Metric(name: "f1", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    labels (`list` of `int`): The set of labels to include when `average` is not set to `'binary'`, and the order of the labels if `average` is `None`. Labels present in the data can be excluded, for example to calculate a multiclass average ignoring a majority negative class. Labels not present in the data will result in 0 components in a macro average. For multilabel targets, labels are column indices. By default, all labels in `predictions` and `references` are used in sorted order. Defaults to None.
    pos_label (`int`): The class to be considered the positive class, in the case where `average` is set to `binary`. Defaults to 1.
    average (`string`): This parameter is required for multiclass/multilabel targets. If set to `None`, the sco

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

**Afinar el modelo pre-entrenado (Fine-Tuning)**

In [ ]:
#model.config.classifier_dropout=0.01
model.config.num_attention_heads=20
model.config.num_hidden_layers=20

In [ ]:
from transformers import TrainingArguments

model_name = model_checkpoint.split("/")[-1]

batch_size = 16
num_train_epochs=2
train_dataset = encoded_dataset["train"]
logging_steps = len(train_dataset) // (2 * batch_size * num_train_epochs)

training_args = TrainingArguments(
    output_dir="results",
    num_train_epochs=num_train_epochs,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=logging_steps,
    push_to_hub=False,
    #push_to_hub_model_id=f"{model_name}-finetuned-MeIA-AnalisisDeSentimientos"
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,1.126900,1.115067,0.472925
2,1.002700,1.014919,0.545852


TrainOutput(global_step=1532, training_loss=1.1437025546405084, metrics={'train_runtime': 1831.3009, 'train_samples_per_second': 13.378, 'train_steps_per_second': 0.837, 'total_flos': 4661155366438908.0, 'train_loss': 1.1437025546405084, 'epoch': 2.0})

**Evaluación del modelo**

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.9402682185173035, 'eval_f1': 0.5946710413910401, 'eval_runtime': 82.2699, 'eval_samples_per_second': 63.814, 'eval_steps_per_second': 2.662, 'epoch': 2.0}


**Subir el modelo a Hugging Face**

In [ ]:
trainer.push_to_hub()

**Pruebas**

Cargar conjunto de datos de pruebas

In [ ]:
import pandas as pd
data = pd.read_excel('/content/drive/Shareddrives/MeIA-Análisis de Sentimiento/test.xlsx', index_col=0)

Cargar pipeline del modelo entrenado

In [ ]:
trainer.save_model('/content/drive/Shareddrives/MeIA-Análisis de Sentimiento/Roberta')

NameError: ignored

In [ ]:
from transformers import AutoModelForSequenceClassification
model2 = AutoModelForSequenceClassification.from_pretrained('/content/drive/Shareddrives/MeIA-Análisis de Sentimiento/Roberta Training Diff')

In [ ]:
from transformers import pipeline
pi = pipeline("text-classification", model=model2,tokenizer=tokenizer, max_length=512, truncation=True)

**Ejemplo de prueba**

In [ ]:
text = "Bueno es un. Hotel que ya es necesario un mantenimiento y una modernización urgente porque con el nivel que esta presentando a hora no le alcanza ni par un 3 estrella sus ascensores son deplorable  y tiene un mal servicio de su personal"
res = pi(text)
print(res)

[{'label': 'LABEL_1', 'score': 0.7145068049430847}]


**Conjunto de pruebas y generción de archivo de texto para reto MeIA**

In [ ]:
#Abrir archivo de texto en modo escritura para almacenar los resultados para el reto
with open('/content/drive/Shareddrives/MeIA-Análisis de Sentimiento/Resultados_MeIATeam_ModeloBase_ReseniaRobertaTrain-2.txt', 'w') as f:
  for i in range(len(data)):
      #print(i," - ",data.iloc[i]['Review'])
      res = pi(data.iloc[i]['Review'])
      label = res[0]['label']
      #Establecer la etiqueta correspondiente
      if label == 'LABEL_0':
        label = "1"
      elif label == 'LABEL_1':
        label = "2"
      elif label == 'LABEL_2':
        label = "3"
      elif label == 'LABEL_3':
        label = "4"
      elif label == 'LABEL_4':
        label = "5"
      #Escribir predicción del sentimiento en el archivo de texto con el formaro correspondiente
      f.write(str(i)+"\t"+label+"\n")
      #Imprimir en pantalla lo que se escribio en el archivo txt
      #print(str(i)+"\t"+label+"\n")
      #Descomente si solo quiere probar este bloque de codigo para las primeras tres instancias de pruebas
      #if i ==2:
        #break